# Import depdendencies
kernel = conda_tensorflow2_p36

In [ ]:
import shutil
import sagemaker
import os
import numpy as np
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlowModel
from sagemaker.tensorflow import TensorFlow
from tensorflow.python.keras.preprocessing.image import load_img

In [2]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [24]:
os.environ['SM_MODEL_DIR'] = "/opt/ml/model"
print(os.environ['SM_MODEL_DIR'])

/opt/ml/model


## S3 bucket URI for training, validation and testing data

In [ ]:
# Set S3 address to training data
bucket = "tickercardiology-echocv-sagemaker"
key = "model-data"
training_data_uri = "s3://{}/{}/train/".format(bucket, key)
# Set S3 address to validation data
validation_data_uri = "s3://{}/{}/test/".format(bucket, key)

## Display training script

In [ ]:
!pygmentize 'keras_model_fn.py'

## Create TensorFlow2 estimator 

In [ ]:
estimator2 = TensorFlow(entry_point='keras_model_fn.py',
                        role=role,
                        train_instance_count=1,
                        train_instance_type='ml.g4dn.xlarge', #$0.958
                        framework_version='2.1.0',
                        py_version='py36',
                        hyperparameters={
                                        'epochs': 15,
                                        'batch_size': 64,
                                        'learning_rate': 1e-5},
                        script_mode=True,
                        model_dir = os.environ['SM_MODEL_DIR'],
                        distributions={'parameter_server': {'enabled': True}})

bucket = "sagemaker-ap-southeast-2-611188727347"
key = "test1"
model_path = "s3://{}/{}/".format(bucket, key)

## Start training model
model save path:
S3://{sagemaker-ap-southeast-2-611188727347}.{tensorflow-training-year-month-data-hour-minute-second-ms}/output/model.tar.gz

In [ ]:
estimator2.fit({'training': training_data_uri, 'validation': validation_data_uri})

## Load model from S3, if estimator trained in this notebook

In [ ]:
predictor2 = estimator2.deploy(initial_instance_count=1, instance_type='ml.m5.2xlarge', endpoint_name='epoch15-3')

# OR
## Load model from S3, if estimator not trained in this notebook 

In [26]:
bucket = "sagemaker-ap-southeast-2-611188727347"
key = "tensorflow-training-2020-11-09-01-37-28-348"
model_path = "s3://{}/{}/output/model.tar.gz".format(bucket, key)

model = TensorFlowModel(model_data=model_path, 
                        role=role,
                        framework_version='2.1.0')

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.2xlarge')

## Delete the endpoints to save resources

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)

In [ ]:
def classify(directory, predictor):
    """
    Classifies echo images in given directory

    @param directory: folder with jpg echo images for classification
    """
    imagedict = {}
    predictions = {}
    for filename in os.listdir(directory):
        if "jpg" in filename:
            image = load_img(directory + filename, target_size=(224, 224))
            imagedict[filename] = np.array(image).reshape((1, 224, 224, 3))
        
    for filename in imagedict:
        result = predictor.predict({'inputs_input': imagedict[filename]})
        predictions[filename] = result["predictions"][0]
    return predictions

In [ ]:
preds = classify("images/test/random/", predictor2)
print(preds)

for p in preds:
    print(preds[p])

In [ ]:
## Download the .pb and variables from S3
# !aws s3 cp s3://sagemaker-ap-southeast-2-611188727347/tensorflow-training-2020-11-06-03-49-07-502/model/1 temp/1 --recursive 
## Run this in terminal:    tar -C "$PWD" -czf model.tar.gz temp/
## Copy the tar.gz to the s3://kate//tensorflow-training-date/output/
# !aws s3 cp model.tar.gz s3://sagemaker-ap-southeast-2-611188727347/tensorflow-training-2020-11-06-03-49-07-502/output/ 